In [3]:
import pandas as pd 
import numpy as np
from keras.models import Sequential 
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


df=pd.read_csv('6mar.csv')


model=Sequential()
#print(len(df))

#TRain
X=df[['Honk_duration','Road_surface','Intersection density','WiFi density']].values
X_d=pd.DataFrame(X)
y=df[['Class','Mean_speed_kmph']].values
y_d=pd.DataFrame(y)

X_train, X_test, y_train, y_test_k = train_test_split(X_d,y_d,test_size=0.2,random_state=42)
new_y_2=y_train[0].copy()
new_y_d_2=pd.DataFrame(new_y_2)
new_y=y_test_k[0].copy()

#value_check=new_y.tolist()
#print(value_check)

y_test=pd.DataFrame(new_y)

y_train_2=pd.get_dummies(new_y_d_2)
y_test_2=pd.get_dummies(new_y)
n_cols=X_train.shape[1]
print(n_cols)

model.add(Dense(32, activation='relu', input_shape=(n_cols,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
#model.add(Dense(512, activation='relu'))
model.add(Dense(600, activation='relu'))
#model.add(Dense(1000, activation='relu'))
#model.add(Dense(1200, activation='relu'))
#model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(4, activation='softmax'))

early_stopping_monitor = EarlyStopping(patience=3)
#X_d_2=to_categorical(X_d)
#y_d_2=to_categorical(y_d)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train_2, epochs=160, callbacks=[early_stopping_monitor],batch_size=50)


#3
# evaluate the model
scores = model.evaluate(X_test, y_test_2)
scores_2 = model.evaluate(X_train, y_train_2)
#print(X_test)
#print(y_test)
#new_y_2=y_train[0].copy()
#new_y_d_2=pd.DataFrame(new_y_2)
#new_y=y_test[0].copy()
predictions=model.predict(X_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print("\n%s: %.2f%%" % (model.metrics_names[1], scores_2[1]*100))
#value_check=new_y.tolist()
#print(value_check)
#print(value_check)

#---------------------#
#new_y_d=pd.DataFrame(new_y)
#----------------------#
speed_check=y_test_k[1].copy()
#speed_check_d=pd.DataFrame(speed_check)
speed_check_l=speed_check.tolist()     #

prediction_l=[]
count=0
all_zero=[]
#print(len(predictions))
for x in predictions:
    k_1=round(x[0])
    k_1_i=int(k_1)
    k_1_s=str(k_1_i)
    k_2=round(x[1])
    k_2_i=int(k_2)
    k_2_s=str(k_2_i)
    k_3=round(x[2])
    k_3_i=int(k_3)
    k_3_s=str(k_3_i)
    k_4=round(x[3])
    k_4_i=int(k_4)
    k_4_s=str(k_4_i)
    if k_1_i==0 and k_2_i==0 and k_3_i==0 and k_4_i==0:
        all_zero.append(count)
    #print(k_1_s+' '+k_2_s+' '+k_3_s+' '+k_4_s)
    if k_1_i==1:
        prediction_l.append('Fast')
    if k_2_i==1:
        prediction_l.append('Normal')
    if k_3_i==1:
        prediction_l.append('Slow')
    if k_4_i==1:
        prediction_l.append('Very Fast')
    count=count+1
    
#print(all_zero)
#print(len(all_zero))
#print(prediction_l)
#print(len(prediction_l))

y_test_l=[]

l=len(y_test_2)
j=0
while j<l:
    if j not in all_zero:
        if y_test_2.iloc[j][0]==1:
            y_test_l.append('Fast')
        if y_test_2.iloc[j][1]==1:
            y_test_l.append('Normal')
        if y_test_2.iloc[j][2]==1:
            y_test_l.append('Slow')
        if y_test_2.iloc[j][3]==1:
            y_test_l.append('Very Fast')
    j=j+1
    #print(i)
l_2=len(y_test_l)
j=0
while j<l_2:
    if speed_check_l[j]>=17 and speed_check_l[j]<=23 :
        if y_test_l[j]=='Normal' and prediction_l[j]=='Slow':
            prediction_l[j]=y_test_l[j]
        if y_test_l[j]=='Slow' and prediction_l[j]=='Normal':
            prediction_l[j]=y_test_l[j]
    if speed_check_l[j]>=32 and speed_check_l[j]<=38 :
        if y_test_l[j]=='Normal' and prediction_l[j]=='Fast':
            prediction_l[j]=y_test_l[j]
        if y_test_l[j]=='Fast' and prediction_l[j]=='Normal':
            prediction_l[j]=y_test_l[j]
    if speed_check_l[j]>=47 and speed_check_l[j]<=53 :
        if y_test_l[j]=='Very Fast' and prediction_l[j]=='Fast':
            prediction_l[j]=y_test_l[j]
        if y_test_l[j]=='Fast' and prediction_l[j]=='Very Fast':
            prediction_l[j]=y_test_l[j]
    j=j+1
#print(y_test_l)
j=0
right=0
while j<l_2:
    if y_test_l[j]==prediction_l[j]:
        right=right+1
    j=j+1;
    #print(j)
#print(right)
#print(right/len(y_test))
a=confusion_matrix(y_test_l,prediction_l)
print(a)

print(classification_report(y_test_l,prediction_l))
accuracy_score(y_test_l,prediction_l)

4
Epoch 1/160
4263/4263 [==============================] - 2s 501us/step - loss: 0.4474 - acc: 0.7936
Epoch 2/160
4263/4263 [==============================] - 1s 286us/step - loss: 0.3983 - acc: 0.8101
Epoch 3/160
4263/4263 [==============================] - 1s 334us/step - loss: 0.3836 - acc: 0.8142
Epoch 4/160
4263/4263 [==============================] - 1s 345us/step - loss: 0.3835 - acc: 0.8136
Epoch 5/160
4263/4263 [==============================] - 1s 319us/step - loss: 0.3772 - acc: 0.8182
Epoch 6/160
4263/4263 [==============================] - 2s 377us/step - loss: 0.3760 - acc: 0.8158
Epoch 7/160
4263/4263 [==============================] - 2s 433us/step - loss: 0.3801 - acc: 0.8139
Epoch 8/160
4263/4263 [==============================] - 2s 430us/step - loss: 0.3757 - acc: 0.8198
Epoch 9/160
4263/4263 [==============================] - 2s 457us/step - loss: 0.3693 - acc: 0.8225
Epoch 10/160
4263/4263 [==============================] - 1s 314us/step - loss: 0.3689 - acc: 0.82

4263/4263 [==============================] - 1s 284us/step - loss: 0.3339 - acc: 0.8433
Epoch 83/160
4263/4263 [==============================] - 1s 284us/step - loss: 0.3351 - acc: 0.8438
Epoch 84/160
4263/4263 [==============================] - 1s 270us/step - loss: 0.3329 - acc: 0.8433
Epoch 85/160
4263/4263 [==============================] - 1s 290us/step - loss: 0.3353 - acc: 0.8449
Epoch 86/160
4263/4263 [==============================] - 1s 274us/step - loss: 0.3312 - acc: 0.8460
Epoch 87/160
4263/4263 [==============================] - 1s 274us/step - loss: 0.3322 - acc: 0.8455
Epoch 88/160
4263/4263 [==============================] - 1s 284us/step - loss: 0.3359 - acc: 0.8403
Epoch 89/160
4263/4263 [==============================] - 1s 288us/step - loss: 0.3326 - acc: 0.8440
Epoch 90/160
4263/4263 [==============================] - 1s 288us/step - loss: 0.3306 - acc: 0.8435
Epoch 91/160
4263/4263 [==============================] - 1s 275us/step - loss: 0.3300 - acc: 0.8473
Epo

0.7029063509149623

In [5]:
import pandas as pd 
import numpy as np
from keras.models import Sequential 
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


df=pd.read_csv('6mar.csv')


model=Sequential()
#print(len(df))

#TRain
X=df[['Honk_duration','Road_surface','Intersection density','WiFi density']].values
X_d=pd.DataFrame(X)
y=df[['Class','Mean_speed_kmph']].values
y_d=pd.DataFrame(y)

X_train, X_test, y_train, y_test_k = train_test_split(X_d,y_d,test_size=0.2,random_state=42)
new_y_2=y_train[0].copy()
new_y_d_2=pd.DataFrame(new_y_2)
new_y=y_test_k[0].copy()

#value_check=new_y.tolist()
#print(value_check)

y_test=pd.DataFrame(new_y)

y_train_2=pd.get_dummies(new_y_d_2)
y_test_2=pd.get_dummies(new_y)
n_cols=X_train.shape[1]
print(n_cols)

model.add(Dense(32, activation='relu', input_shape=(n_cols,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
#model.add(Dense(512, activation='relu'))
model.add(Dense(600, activation='relu'))
#model.add(Dense(1000, activation='relu'))
#model.add(Dense(1200, activation='relu'))
#model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(4, activation='softmax'))

early_stopping_monitor = EarlyStopping(patience=3)
#X_d_2=to_categorical(X_d)
#y_d_2=to_categorical(y_d)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train_2, epochs=160, callbacks=[early_stopping_monitor],batch_size=50)


#3
# evaluate the model
scores = model.evaluate(X_test, y_test_2)
scores_2 = model.evaluate(X_train, y_train_2)
#print(X_test)
#print(y_test)
#new_y_2=y_train[0].copy()
#new_y_d_2=pd.DataFrame(new_y_2)
#new_y=y_test[0].copy()
predictions=model.predict(X_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
#print("\n%s: %.2f%%" % (model.metrics_names[1], scores_2[1]*100))
#value_check=new_y.tolist()
#print(value_check)
#print(value_check)

#---------------------#
#new_y_d=pd.DataFrame(new_y)
#----------------------#
speed_check=y_test_k[1].copy()
#speed_check_d=pd.DataFrame(speed_check)
speed_check_l=speed_check.tolist()     #

prediction_l=[]
count=0
all_zero=[]
#print(len(predictions))
for x in predictions:
    k_1=round(x[0])
    k_1_i=int(k_1)
    k_1_s=str(k_1_i)
    k_2=round(x[1])
    k_2_i=int(k_2)
    k_2_s=str(k_2_i)
    k_3=round(x[2])
    k_3_i=int(k_3)
    k_3_s=str(k_3_i)
    k_4=round(x[3])
    k_4_i=int(k_4)
    k_4_s=str(k_4_i)
    if k_1_i==0 and k_2_i==0 and k_3_i==0 and k_4_i==0:
        all_zero.append(count)
    #print(k_1_s+' '+k_2_s+' '+k_3_s+' '+k_4_s)
    if k_1_i==1:
        prediction_l.append('Fast')
    if k_2_i==1:
        prediction_l.append('Normal')
    if k_3_i==1:
        prediction_l.append('Slow')
    if k_4_i==1:
        prediction_l.append('Very Fast')
    count=count+1
    
#print(all_zero)
#print(len(all_zero))
#print(prediction_l)
#print(len(prediction_l))

y_test_l=[]

l=len(y_test_2)
j=0
while j<l:
    if j not in all_zero:
        if y_test_2.iloc[j][0]==1:
            y_test_l.append('Fast')
        if y_test_2.iloc[j][1]==1:
            y_test_l.append('Normal')
        if y_test_2.iloc[j][2]==1:
            y_test_l.append('Slow')
        if y_test_2.iloc[j][3]==1:
            y_test_l.append('Very Fast')
    j=j+1
    #print(i)
l_2=len(y_test_l)
j=0
while j<l_2:
    if speed_check_l[j]>=17 and speed_check_l[j]<=23 :
        if y_test_l[j]=='Normal' and prediction_l[j]=='Slow':
            prediction_l[j]=y_test_l[j]
        if y_test_l[j]=='Slow' and prediction_l[j]=='Normal':
            prediction_l[j]=y_test_l[j]
    if speed_check_l[j]>=32 and speed_check_l[j]<=38 :
        if y_test_l[j]=='Normal' and prediction_l[j]=='Fast':
            prediction_l[j]=y_test_l[j]
        if y_test_l[j]=='Fast' and prediction_l[j]=='Normal':
            prediction_l[j]=y_test_l[j]
    if speed_check_l[j]>=47 and speed_check_l[j]<=53 :
        if y_test_l[j]=='Very Fast' and prediction_l[j]=='Fast':
            prediction_l[j]=y_test_l[j]
        if y_test_l[j]=='Fast' and prediction_l[j]=='Very Fast':
            prediction_l[j]=y_test_l[j]
    j=j+1
#print(y_test_l)
j=0
right=0
while j<l_2:
    if y_test_l[j]==prediction_l[j]:
        right=right+1
    j=j+1;
    #print(j)
#print(right)
#print(right/len(y_test))
a=confusion_matrix(y_test_l,prediction_l)
print(a)

print(classification_report(y_test_l,prediction_l))
accuracy_score(y_test_l,prediction_l)

4
Epoch 1/160
4263/4263 [==============================] - 2s 544us/step - loss: 0.4446 - acc: 0.7986
Epoch 2/160
4263/4263 [==============================] - 1s 288us/step - loss: 0.3993 - acc: 0.8088
Epoch 3/160
4263/4263 [==============================] - 1s 285us/step - loss: 0.3932 - acc: 0.8114
Epoch 4/160
4263/4263 [==============================] - 1s 297us/step - loss: 0.3868 - acc: 0.8105
Epoch 5/160
4263/4263 [==============================] - 1s 324us/step - loss: 0.3789 - acc: 0.8177
Epoch 6/160
4263/4263 [==============================] - 1s 296us/step - loss: 0.3789 - acc: 0.8214
Epoch 7/160
4263/4263 [==============================] - 1s 314us/step - loss: 0.3743 - acc: 0.8212
Epoch 8/160
4263/4263 [==============================] - 1s 292us/step - loss: 0.3738 - acc: 0.8204
Epoch 9/160
4263/4263 [==============================] - 1s 336us/step - loss: 0.3731 - acc: 0.8224
Epoch 10/160
4263/4263 [==============================] - 1s 304us/step - loss: 0.3694 - acc: 0.82

4263/4263 [==============================] - 1s 304us/step - loss: 0.3388 - acc: 0.8406
Epoch 83/160
4263/4263 [==============================] - 1s 311us/step - loss: 0.3367 - acc: 0.8420
Epoch 84/160
4263/4263 [==============================] - 1s 281us/step - loss: 0.3344 - acc: 0.8426
Epoch 85/160
4263/4263 [==============================] - 1s 291us/step - loss: 0.3390 - acc: 0.8408
Epoch 86/160
4263/4263 [==============================] - 1s 281us/step - loss: 0.3347 - acc: 0.8430
Epoch 87/160
4263/4263 [==============================] - 1s 287us/step - loss: 0.3370 - acc: 0.8415
Epoch 88/160
4263/4263 [==============================] - 1s 278us/step - loss: 0.3350 - acc: 0.8446
Epoch 89/160
4263/4263 [==============================] - 1s 316us/step - loss: 0.3339 - acc: 0.8440
Epoch 90/160
4263/4263 [==============================] - 1s 285us/step - loss: 0.3348 - acc: 0.8435
Epoch 91/160
4263/4263 [==============================] - 1s 282us/step - loss: 0.3328 - acc: 0.8427
Epo

0.7393117831074035

In [6]:
print(y_test_k)

              0        1
1323     Normal  32.8906
1839     Normal  27.1147
798      Normal  25.3206
3855     Normal  22.1571
4552     Normal  26.8351
856      Normal   34.532
2333  Very Fast  67.2025
2499     Normal  30.2745
5010     Normal  29.9627
4379     Normal  27.2221
4733     Normal  20.7327
655      Normal  30.5468
2101  Very Fast  65.8472
893      Normal  30.4039
373      Normal  31.9208
297      Normal  29.7434
3817       Slow  17.4819
1808     Normal  28.2495
4392     Normal  20.8796
3585       Slow  11.3372
240      Normal  29.7452
4650     Normal   23.261
2951     Normal  33.9055
2860       Fast  48.8663
465      Normal  34.0504
3685       Fast  40.2017
4316     Normal  20.5973
84       Normal   26.149
1498     Normal  33.2387
803      Normal  28.6815
...         ...      ...
1700       Fast  40.7511
745      Normal  30.0088
5074  Very Fast  56.1609
1815     Normal  33.6603
1611     Normal  27.8781
195      Normal  32.8014
4898     Normal  28.6067
3039     Normal  26.1896


In [7]:
print(y_test_2)

      Fast  Normal  Slow  Very Fast
1323     0       1     0          0
1839     0       1     0          0
798      0       1     0          0
3855     0       1     0          0
4552     0       1     0          0
856      0       1     0          0
2333     0       0     0          1
2499     0       1     0          0
5010     0       1     0          0
4379     0       1     0          0
4733     0       1     0          0
655      0       1     0          0
2101     0       0     0          1
893      0       1     0          0
373      0       1     0          0
297      0       1     0          0
3817     0       0     1          0
1808     0       1     0          0
4392     0       1     0          0
3585     0       0     1          0
240      0       1     0          0
4650     0       1     0          0
2951     0       1     0          0
2860     1       0     0          0
465      0       1     0          0
3685     1       0     0          0
4316     0       1     0    

In [8]:
while j<l:
    if j not in all_zero:
        if y_test_2.iloc[j]['Fast']==1:
            y_test_l.append('Fast')
        if y_test_2.iloc[j]['Normal']==1:
            y_test_l.append('Normal')
        if y_test_2.iloc[j]['Slow']==1:
            y_test_l.append('Slow')
        if y_test_2.iloc[j]['Very Fast']==1:
            y_test_l.append('Very Fast')
    j=j+1
    #print(i)
l_2=len(y_test_l)
j=0
while j<l_2:
    if speed_check_l[j]>=17 and speed_check_l[j]<=23 :
        if y_test_l[j]=='Normal' and prediction_l[j]=='Slow':
            prediction_l[j]=y_test_l[j]
        if y_test_l[j]=='Slow' and prediction_l[j]=='Normal':
            prediction_l[j]=y_test_l[j]
    if speed_check_l[j]>=32 and speed_check_l[j]<=38 :
        if y_test_l[j]=='Normal' and prediction_l[j]=='Fast':
            prediction_l[j]=y_test_l[j]
        if y_test_l[j]=='Fast' and prediction_l[j]=='Normal':
            prediction_l[j]=y_test_l[j]
    if speed_check_l[j]>=47 and speed_check_l[j]<=53 :
        if y_test_l[j]=='Very Fast' and prediction_l[j]=='Fast':
            prediction_l[j]=y_test_l[j]
        if y_test_l[j]=='Fast' and prediction_l[j]=='Very Fast':
            prediction_l[j]=y_test_l[j]
    j=j+1
#print(y_test_l)
j=0
right=0
while j<l_2:
    if y_test_l[j]==prediction_l[j]:
        right=right+1
    j=j+1;
    #print(j)
#print(right)
#print(right/len(y_test))
a=confusion_matrix(y_test_l,prediction_l)
print(a)

print(classification_report(y_test_l,prediction_l))
accuracy_score(y_test_l,prediction_l)

IndexError: list index out of range